This notebook contains the necessary scripts for making the SMOGN model.

In [1]:
'''Starting with the import of the necessary libraries including 
the smogn (https://github.com/nickkunz/smogn/tree/master):'''
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import scipy
from scipy.stats import linregress
import seaborn as sns
import warnings
import smogn
warnings.filterwarnings('ignore')

C:\Users\po7517\Anaconda3\envs\new_env\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\po7517\Anaconda3\envs\new_env\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\po7517\Anaconda3\envs\new_env\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
### either CPU or GPU:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [3]:
### Reading the dummy data
data = pd.read_csv('data/all_data_dummy.csv')
### Applying the SMOGN tranformation on the data
target_variables = ['B', 'V', 'Cr', 'Mn', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Be',
                    'As', 'Rb', 'Sr', 'Y', 'Mo', 'Cd', 'Sb', 'Cs', 'Ba', 'Pb']
input_features = ['Li', 'Na', 'Ca', 'SO-', 'Cl-', 'Br-', 'Al', 'Si', 'pH', 'F-', 'Fe', 'Mg']
raw_data = data[target_variables+input_features]

In [4]:
def apply_smogn_safe(data, target_col):
    '''
    This function runs the SMOGN algorithm. 
    For details of SMOGN see https://github.com/nickkunz/smogn/tree/master.
    '''
    try:
        smogn_data = smogn.smoter(
            data=data,
            y=target_col
        )
        return smogn_data, True
    except Exception as e:
        print(f"SMOGN failed: {e}\n→ Using original data instead.")
        return data, False

def pytorch_NN(raw_data, input_features, target_variables, smogn_target='V', weight_decay=1e-3, patience=100):
    '''
    This function reads the data and based on the interest of the user, different 
    input features and target variables can be selected from the existing measured
    parameters.
    weight_decay is a regularization technique used to prevent overfitting by penalizing 
    large weights in the model. More details at 
    https://pytorch.org/docs/stable/generated/torch.optim.Adam.html
    patience refers to the number of training epochs to wait after the last improvement 
    in the monitored metric (e.g., validation loss, accuracy) before stopping the training process.
    More details at https://pytorch.org/ignite/generated/ignite.handlers.early_stopping.EarlyStopping.html
    '''
    ### Step 1: Combine input and target
    X = raw_data[input_features]
    y = raw_data[target_variables]
    full_data = pd.concat([X, y], axis=1)

    ### Step 2: Split into train (60%), val (20%), test (20%)
    train_val_data, test_data = train_test_split(full_data, test_size=0.2)
    train_data, val_data = train_test_split(train_val_data, test_size=0.25)  # 0.25 * 0.8 = 0.2

    ### Step 3: Apply SMOGN on training data only
    train_data = train_data.reset_index(drop=True)
    train_data_smogn, smogn_success = apply_smogn_safe(train_data, smogn_target)

    ### Step 4: Split X and y again
    X_train = train_data_smogn[input_features].to_numpy()
    y_train = train_data_smogn[target_variables].to_numpy()

    X_val = val_data[input_features].to_numpy()
    y_val = val_data[target_variables].to_numpy()

    X_test = test_data[input_features].to_numpy()
    y_test = test_data[target_variables].to_numpy()

    ### Step 5: Normalize
    scaler_x = RobustScaler()
    scaler_y = RobustScaler()

    X_train_norm = scaler_x.fit_transform(X_train)
    X_val_norm = scaler_x.transform(X_val)
    X_test_norm = scaler_x.transform(X_test)

    y_train_norm = scaler_y.fit_transform(y_train)
    y_val_norm = scaler_y.transform(y_val)
    y_test_norm = scaler_y.transform(y_test)

    ### Step 6: Convert to tensors
    XX = torch.from_numpy(X_train_norm.astype(np.float32)).to(device)
    yy = torch.from_numpy(y_train_norm.astype(np.float32)).to(device)
    XX_val = torch.from_numpy(X_val_norm.astype(np.float32)).to(device)
    yy_val = torch.from_numpy(y_val_norm.astype(np.float32)).to(device)

    input_size = XX.shape[1]
    output_size = yy.shape[1]

    ### Step 7: Define the model
    class NN(nn.Module):
        def __init__(self):
            super(NN, self).__init__()
            self.fc1 = nn.Linear(input_size, 32)
            self.norm1 = nn.BatchNorm1d(32)
            self.fc2 = nn.Linear(32, 16)
            self.norm2 = nn.BatchNorm1d(16)
            self.fc3 = nn.Linear(16, output_size)

        def forward(self, x):
            x = torch.relu(self.norm1(self.fc1(x)))
            x = torch.relu(self.norm2(self.fc2(x)))
            return self.fc3(x)

    model = NN().to(device)
    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(), lr=0.1, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=patience, verbose=True)

    ### Step 8: Training with early stopping
    best_loss = float('inf')
    patience_counter = 0
    num_epochs = 2000
    loss_train = []
    loss_val = []

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(XX)
        loss = criterion(outputs, yy)
        loss.backward()
        optimizer.step()
        loss_train.append(loss.item())

        model.eval()
        with torch.no_grad():
            val_outputs = model(XX_val)
            val_loss = criterion(val_outputs, yy_val)
            loss_val.append(val_loss.item())

        scheduler.step(val_loss)

        if val_loss.item() < best_loss:
            best_loss = val_loss.item()
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

        if (epoch+1) % 100 == 0:
            print(f"Epoch {epoch+1}, Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

    print("Training complete.")

    ### Step 9: Evaluate on test set
    model.eval()
    with torch.no_grad():
        X_test_tensor = torch.from_numpy(X_test_norm.astype(np.float32)).to(device)
        pred_test = model(X_test_tensor).cpu().numpy()
        pred_test_orig = scaler_y.inverse_transform(pred_test)
        y_test_orig = scaler_y.inverse_transform(y_test_norm)

        pred_train = model(XX).cpu().numpy()
        pred_train_orig = scaler_y.inverse_transform(pred_train)
        y_train_orig = scaler_y.inverse_transform(y_train_norm)

    ### Step 10: R² values
    r_values = []
    for i in range(len(target_variables)):
        r = linregress(y_test_orig[:, i], pred_test_orig[:, i]).rvalue
        r_values.append(r)

    return r_values, y_test_orig, pred_test_orig, y_train_orig, pred_train_orig, loss_train, loss_val

In [6]:
target_variables = ['B', 'V', 'Cr', 'Mn', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Be',
                    'As', 'Rb', 'Sr', 'Y', 'Mo', 'Cd', 'Sb', 'Cs', 'Ba', 'Pb'] # to be predicted
input_features = ['Li', 'Na', 'Ca', 'SO-', 'Cl-', 'Br-', 'Al', 'Si', 'pH', 'F-', 'Fe', 'Mg']
import time
start = time.time()
results = []
### Making an ensemble of 1000 models
for i in range(100):
    print(f'Starting Round {i}')
    r_values, test_y, predicted_y, train_y, predicted_y_train, l_tr, l_te = pytorch_NN(raw_data, input_features, target_variables)
    results.append({
        'r_values': r_values,
        'test_y': test_y,
        'predicted_y': predicted_y,
        'train_y': train_y,
        'predicted_y_train': predicted_y_train,
        'l_tr': l_tr,
        'l_te': l_te,
    })
end = time.time()
print(f'{(end - start)/60} mins')

Starting Round 0


r_index: 100%|########################################################################| 10/10 [00:00<00:00, 123.14it/s]


Epoch 100, Train Loss: 0.1246, Val Loss: 0.3214
Epoch 200, Train Loss: 0.1110, Val Loss: 0.3120
Epoch 300, Train Loss: 0.1130, Val Loss: 0.3056
Early stopping at epoch 331
Training complete.
Starting Round 1


r_index: 100%|########################################################################| 10/10 [00:00<00:00, 112.71it/s]


Epoch 100, Train Loss: 0.1200, Val Loss: 0.3083
Epoch 200, Train Loss: 0.0982, Val Loss: 0.2847
Early stopping at epoch 230
Training complete.
Starting Round 2


...

Epoch 100, Train Loss: 0.1210, Val Loss: 0.3189
Epoch 200, Train Loss: 0.1064, Val Loss: 0.2922
Epoch 300, Train Loss: 0.1066, Val Loss: 0.3148
Early stopping at epoch 344
Training complete.
Starting Round 98


r_index: 100%|#########################################################################| 10/10 [00:00<00:00, 77.15it/s]


Epoch 100, Train Loss: 0.1551, Val Loss: 0.3456
Epoch 200, Train Loss: 0.1191, Val Loss: 0.3290
Epoch 300, Train Loss: 0.1110, Val Loss: 0.3105
Early stopping at epoch 342
Training complete.
Starting Round 99


r_index: 100%|########################################################################| 10/10 [00:00<00:00, 104.37it/s]


Epoch 100, Train Loss: 0.1196, Val Loss: 0.2990
Early stopping at epoch 181
Training complete.
1.5669477303822836 mins


In [ ]:
l_te = [[r for r in result['l_te']] for result in results]
l_tr = [[r for r in result['l_tr']] for result in results]
fig, ax = plt.subplots(figsize=(4, 4))
for i in l_tr:
    ax.plot(i, c='b', lw=0.5, label='Train')
    ax.set_yscale('log')
for i in l_te:
    ax.plot(i, c='r', lw=0.5, label='Test')
    ax.set_yscale('log')
ax.set_xlabel('Epochs', fontsize=10, family='Arial')
ax.set_ylabel('MAE', fontsize=10, family='Arial')
plt.yticks(fontname = "Arial", fontsize=10)
plt.xticks(fontname = "Arial", fontsize=10)
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), fontsize='small')
plt.show()

In [ ]:
### Separating the R2 score of each model
r_squared_values = [[r**2 for r in result['r_values']] for result in results]
df = pd.DataFrame(r_squared_values, columns=target_variables)
df_melted = df.melt(var_name='Element', value_name='R_squared')

In [ ]:
### Making a plot to have a view on the R2 score for each predicted element
fig, ax = plt.subplots(1,1, figsize=(8,3))
sns.boxplot(data=df_melted, x='Element', y='R_squared')
plt.xlabel('Elements', fontname = "Arial", fontsize=10)
plt.ylabel('R$^2$', fontname = "Arial", fontsize=10)
plt.xticks(rotation=90, ha='right')
plt.show()